# Stimulants and Fentanyl

**The goal of this analysis is to characterize the presence of fentanyl in street drugs expected to be stimulants (methamphetaine, cocaine).**<br><br>
Drug samples were obtained by the [UNC Drug Checking Lab](https://streetsafe.supply)<br>

Dataset used in this analysis is publicly available with DOI [10.17615/85h3-p995](https://doi.org/10.17615/85h3-p995).<br>
The python code used in the analysis were generated by ChatGPT from Open AI, under the supervision of an analyst.<br><br>
[Laboratory analysis methods](https://github.com/opioiddatalab/drugchecking/blob/main/docs/lab_methods.md) are available here.

In [2]:
import pandas as pd

# Import data
url = "https://raw.githubusercontent.com/opioiddatalab/drugchecking/main/datasets/selfservice/expected_stimulants/stimfent.csv"
df = pd.read_csv(url)

# Convert the 'date_collect' column to a Pandas datetime format
df['date_collect'] = pd.to_datetime(df['date_collect'])

# Convert the 'date_complete' column to a Pandas datetime format
df['date_complete'] = pd.to_datetime(df['date_complete'])
df['date_complete'] = df['date_complete'].dt.date

TypeError: required field "type_ignores" missing from Module

# Sample Descriptives
---

## Sample Size & Dates
---

**A total of N=178 samples expected to be crack/cocaine or methamphetamine were analyzed using GCMS.
<br>Samples were collected between May 5, 2021 and January 27, 2023.
<br>Laboratory analysis took place between January 27, 2022 and February 1, 2023.**

In [2]:
samplesize = df['sampleid'].nunique()
print("Number of unique samples expected to be a stimulants:", samplesize)

Number of unique samples expected to be a stimulants: 178


In [3]:
latest_date = df['date_collect'].max()
earliest_date = df['date_collect'].min()

print("Date range for sample collection")
print("Earliest collection:", earliest_date)
print("Latest collection:", latest_date)

Date range for sample collection
Earliest collection: 2021-05-05 00:00:00
Latest collection: 2023-01-27 00:00:00


In [4]:
latest_date = df['date_complete'].max()
earliest_date = df['date_complete'].min()

print("Date range for sample processing")
print("Earliest sample processing:", earliest_date)
print("Latest sample processing:", latest_date)

Date range for sample processing
Earliest sample processing: 2022-01-27
Latest sample processing: 2023-02-01


## Sample origins
---

**Samples were provided by 31 harm reduction programs or drug user unions in 14 states: North Carolina (n=99), New York (n=22), Tennessee (n=16), Washington (n=11), California (n=10), Oregon (n=4), Texas (n=3), New Mexico (n=3), Arizona (n=2), Michigan (n=2), Ohio (n=2), Nevada (n=2), South Carolina (n=1), and Montana (n=1).**

In [5]:
num_distinct_obs = df['state'].nunique()
print("Number of unique values of state:", num_distinct_obs)

Number of unique values of state: 14


In [6]:
unique_values = df['full_state'].unique()
unique_values_list = unique_values.tolist()
print("Unique values of state':", unique_values_list)

Unique values of state': ['North Carolina', 'South Carolina', 'New York', 'Texas', 'Oregon', 'Arizona', 'Michigan', 'Montana', 'New Mexico', 'Washington', 'Tennessee', 'Ohio', 'Nevada', 'California']


In [7]:
value_counts = df['state'].value_counts()
print("Number of observations for each unique value of state:")
print(value_counts)

Number of observations for each unique value of state:
NC    99
NY    22
TN    16
WA    11
CA    10
OR     4
TX     3
NM     3
AZ     2
MI     2
OH     2
NV     2
SC     1
MT     1
Name: state, dtype: int64


In [8]:
num_distinct_obs = df['program_mask'].nunique()
print("Number of originating harm reduction programs:", num_distinct_obs)

Number of originating harm reduction programs: 31


## Collection Methods
---

**Samples were collected as powder (57%), residue swab (25%), used cottons (6.2%), or from syringe (n=1), with n=19 unknown.**

In [9]:
counts = df['collection'].value_counts()
percents = df['collection'].value_counts(normalize=True)
table = pd.concat([counts, percents], axis=1)
table.columns = ['Counts', 'Percentages']
print(table)

         Counts  Percentages
spatula     102     0.573034
swab         45     0.252809
unknown      19     0.106742
cotton       11     0.061798
syringe       1     0.005618


## Sample Expectations
---

**Prior to lab analysis, sample donors expected 115 to contain methamphetamine, and 63 were expected to be crack/cocaine. Only 19/178 were expected to contain opioids, of which 15 were expected to specifically contain fentanyl. Similar proptions of cocaine (9.5%) and methamphetamine (11.3%) samples were expected to also contain opioids (chi-sq 0.01, df 1, p = 0.91).**

In [10]:
count = df[df['expect_cocaine'] == 1].groupby('sampleid').nunique().shape[0]
print("Expected crack/cocaine samples:")
print(count)

Expected crack/cocaine samples:
63


In [11]:
count = df[df['expect_meth'] == 1].groupby('sampleid').nunique().shape[0]
print("Expected methamphetamine samples:")
print(count)

Expected methamphetamine samples:
115


In [12]:
count = df[df['expect_opioid'] == 1].groupby('sampleid').nunique().shape[0]
print("Expected opioid samples:")
print(count)
count = df[df['expect_fentanyl'] == 1].groupby('sampleid').nunique().shape[0]
print("Expected fentanyl samples:")
print(count)

Expected opioid samples:
19
Expected fentanyl samples:
15


---

*Now we test if cocaine or meth samples were more likely to be considered to also contain opioids.*

In [13]:
# Create variable that is either meth or cocaine, based on sample donor expectations
import numpy as np

df['stimkind'] = np.where((df['expect_meth'] == 1) & (df['expect_cocaine'] == 1), 'Meth & Cocaine',
                           np.where((df['expect_meth'] == 1), 'Meth',
                           np.where((df['expect_cocaine'] == 1), 'Cocaine', 'None')))
counts = df['stimkind'].value_counts()
print(counts)

Meth       115
Cocaine     63
Name: stimkind, dtype: int64


In [14]:
table = pd.crosstab(index=df['stimkind'], columns=df['expect_fentanyl'], margins=True)

print("2x2 Contingency Table:")
table.style.background_gradient(cmap='YlGn')

2x2 Contingency Table:


expect_fentanyl,0,1,All
stimkind,,,
Cocaine,59,4,63
Meth,104,11,115
All,163,15,178


In [15]:
table = pd.crosstab(index=df['stimkind'], columns=df['expect_opioid'], margins=True, normalize='index') * 100
print(table)

expect_opioid          0          1
stimkind                           
Cocaine        90.476190   9.523810
Meth           88.695652  11.304348
All            89.325843  10.674157


In [16]:
from scipy.stats import chi2_contingency

table = pd.crosstab(index=df['stimkind'], columns=df['expect_opioid'])
chi2, p, dof, expected = chi2_contingency(table)
print("Chi-square test statistic:", chi2)
print("p-value:", p)
print("df:", dof)

Chi-square test statistic: 0.013012204195659125
p-value: 0.9091814720272366
df: 1


# Methamphetamine (n=115)
---

**Of 115 samples expected to be methamphetamine, 93.0% (n=107) were confirmed by laboratory analysis to contain methamphetamine as a primary constiuient. Of the 115 expected methamphetamine samples, 14 were found to contain any traces of fentanyl, and 11 had fentanyl as a primary constiuent.**

*Were expected meth samples really meth?*

In [17]:
mask = (df['stimkind'] == 'Meth') & (df['lab_meth_any'] == 1)
count = df[mask]['sampleid'].nunique()
print("Meth samples with any methamphetamine detected:", count)

Meth samples with any methamphetamine detected: 107


In [18]:
mask = (df['stimkind'] == 'Meth') & (df['lab_meth_any'] == 1)
count = df[mask]['sampleid'].nunique()
print("Meth samples with methamphetamine as a primary constiuent:", count)

Meth samples with methamphetamine as a primary constiuent: 107


*So, limit to meth 107 samples that actually turned out to be meth.*

In [19]:
mask = (df['stimkind'] == 'Meth') & (df['lab_meth_any'] == 1)
df_meth_lab = df[mask]
unique_sampleid = df_meth_lab['sampleid'].nunique()
print("Logic check for subsetting meth-expected + meth-positive:", unique_sampleid)

Logic check for subsetting meth-expected + meth-positive: 107


In [20]:
mask = (df['stimkind'] == 'Meth') & (df['lab_fentanyl_any'] == 1)
df_meth_lab = df[mask]
unique_sampleid = df_meth_lab['sampleid'].nunique()
print("Expected meth samples with ANY fentanyl detected:", unique_sampleid)

Expected meth samples with ANY fentanyl detected: 14


In [21]:
mask = (df['stimkind'] == 'Meth') & (df['lab_fentanyl'] == 1)
df_meth_lab = df[mask]
unique_sampleid = df_meth_lab['sampleid'].nunique()
print("Expected meth samples with fentanyl as a primary constiuent:", unique_sampleid)

Expected meth samples with fentanyl as a primary constiuent: 11


## Overdose involvement
---

**Data on overdose involvement was available for 79% (91/115) of expected-methamphetamine samples. One overdose was reported, out of 12 samples, where fentanyl was also found. By comparison, overdose was reported in 5/79 expected-methamphetamine samples where fentanyl was not found. This difference was not statistically significant (chi-sq 78, df 1, p=0.99).**

In [22]:
df_meth = df[df['stimkind'] == "Meth"]
od_crosstab = pd.crosstab(index=df_meth['od'], columns=df_meth['lab_fentanyl_any'], margins=True)
print(od_crosstab)

_, p, _, _ = chi2_contingency(od_crosstab)

print("Chi-square test statistic:", chi2)
print("p-value:", p)
print("df:", dof)

lab_fentanyl_any    0   1  All
od                            
Overdose reported   5   1    6
not involved       74  11   85
All                79  12   91
Chi-square test statistic: 0.013012204195659125
p-value: 0.9994358068766825
df: 1


## Sensations Reported
---

**Among the 14 expected-methamphetamine samples that also contained fentanyl, with 4 samples were reported as feeling  "normal,"  one "stronger" and two "weaker." Two samples were described as "weird", and three were described as "more up", while one was described as "more down." Two samples were remarked as being "nice." No burning sensations, hallucinations, seizures, or skin issues were noted in free text descriptions.**

In [23]:
df_meth_fentanyl = df[(df['stimkind'] == 'Meth') & (df['lab_fentanyl_any'] == 1)]
df_meth_fentanyl[['sampleid', 'sensations', 'sen_strength', 'sen_weird', 'sen_hall', 'sen_up', 'sen_nice', 'sen_down', 'sen_long', 'sen_burn', 'sen_skin', 'sen_seizure']]

,sampleid,sensations,sen_strength,sen_weird,sen_hall,sen_up,sen_nice,sen_down,sen_long,sen_burn,sen_skin,sen_seizure
10,300207,more up,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
29,300385,nice,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
39,300431,normal; nice,normal,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
49,300455,normal,normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,300507,stronger; more up,stronger,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
73,300579,weird,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
87,300783,mixed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,300878,normal,normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,300901,weaker,weaker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,400101,weaker,weaker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Physical description
---

**The 15 expected-methamphetamine samples that contained any fentanyl, were in both cystal (n=5) and powder (n=9) forms. Importantly, fentnayl was present as a primary constiuent in 3/5 cyrstal samples and 7/9 powder samples. The colors described were white (n=6), clear (n=3), yellow (n=2), tan (n=1), and pink (n=1). Collectively, these observations suggest heterogeneous pathways by which fentanyl becomes co-mingled with methamphetamine, and that physical properties are not a reliable marker of the presence of fentanyl.**

In [24]:
df_meth_fentanyl[['sampleid', 'crystals', 'powder', 'lustre', 'color', 'bright_color', 'lab_fentanyl', 'lab_fentanyl_any']]

,sampleid,crystals,powder,lustre,color,bright_color,lab_fentanyl,lab_fentanyl_any
10,300207,NaN,1.0,NaN,white,0,1,1
29,300385,NaN,1.0,NaN,tan,0,1,1
39,300431,1.0,NaN,NaN,clear,0,1,1
49,300455,1.0,NaN,NaN,clear,0,0,1
63,300507,1.0,1.0,NaN,white,0,1,1
73,300579,NaN,1.0,NaN,white,0,1,1
87,300783,NaN,1.0,NaN,white,0,0,1
99,300878,NaN,1.0,NaN,pink,1,1,1
104,300901,1.0,1.0,NaN,clear; white,0,0,1
109,400101,1.0,1.0,NaN,yellow,1,1,1


## Geograhic distribution
---

**Fentanyl-containing methamphetamine samples were observed in North Carolina, Texas, New York, Montana, and Ohio.**

In [56]:
value_counts = df_meth_fentanyl['state'].value_counts()
print("States where methamphetamine-fentanyl was detected:")
print(value_counts)

States where methamphetamine-fentanyl was detected:
NC    10
TX     1
NY     1
MT     1
OH     1
Name: state, dtype: int64


## Text Comments
---

**In free text comments, there was evidence that the sample donor was aware that methamphetamine and fentanyl were mixed together, with one calling it a "speedball," and another precisely noting "meth + fent mixed together. Meth was weaker."**

In [59]:
df_meth_fentanyl[['sampleid', 'texture_notes', 'sensation_notes']]

,sampleid,texture_notes,sensation_notes
10,300207,NaN,NaN
29,300385,NaN,NaN
39,300431,NaN,NaN
49,300455,NaN,NaN
63,300507,NaN,speedball
73,300579,NaN,"fell asleep sitting up, slept for days"
87,300783,NaN,NaN
99,300878,NaN,NaN
104,300901,NaN,NaN
109,400101,NaN,meth + fent mixed together. Meth was weaker


# Cocaine (n=63)
---

**Of 63 samples expected to be cocaine, 90.5% (n=57) were confirmed by laboratory analysis to contain cocaine as a primary constiuient. Of the 63 expected cocaine samples, 9 were found to contain any traces of fentanyl, of which 7 had fentanyl as a primary constiuent.**

*Were cocaine samples really cocaine?*

In [49]:
df_cocaine = df[df["expect_cocaine"] == 1]
value_counts = df_cocaine['lab_cocaine_any'].value_counts()
print("Expected cocaine samples with lab confirmed cocaine as a primary consituent:")
print(value_counts)

Expected cocaine samples with lab confirmed cocaine as a primary consituent:
1    59
0     4
Name: lab_cocaine_any, dtype: int64


*Fentanyl comingling*

In [50]:
df_cocaine = df[df["expect_cocaine"] == 1]
value_counts = df_cocaine['lab_cocaine'].value_counts()
print("Expected cocaine samples with lab confirmed cocaine in trace or primary quantities:")
print(value_counts)

Expected cocaine samples with lab confirmed cocaine in trace or primary quantities:
1    57
0     6
Name: lab_cocaine, dtype: int64


In [54]:
df_cocaine = df[df["expect_cocaine"] == 1]
value_counts = df_cocaine['lab_fentanyl_any'].value_counts()
print("Expected cocaine samples with lab confirmed fentanyl in trace or primary quantity:")
print(value_counts)

Expected cocaine samples with lab confirmed fentanyl in trace or primary quantity:
0    54
1     9
Name: lab_fentanyl_any, dtype: int64


In [53]:
df_cocaine = df[df["expect_cocaine"] == 1]
value_counts = df_cocaine['lab_fentanyl'].value_counts()
print("Expected cocaine samples with lab confirmed fentanyl as primary constituent:")
print(value_counts)

Expected cocaine samples with lab confirmed fentanyl as primary constituent:
0    56
1     7
Name: lab_fentanyl, dtype: int64


## Overdose involvement
---

**Data on overdose involvement was available for 67.7% (42/63) of suspected-cocaine samples. One overdose was reported, out of 5 samples where fentanyl was also found. By comparison, overdose was reported in 2/37 suspected-cocaine samples where fentanyl was not absent. Outcome sparcity precludes statistical testing, and the observed imbalance may be due to random change.**

In [51]:
df_cocaine = df[df['stimkind'] == "Cocaine"]
od_crosstab = pd.crosstab(index=df_cocaine['od'], columns=df_cocaine['lab_fentanyl_any'], margins=True)
print(od_crosstab)

_, p, _, _ = chi2_contingency(od_crosstab)

print("Chi-square test statistic:", chi2)
print("p-value:", p)
print("df:", dof)

lab_fentanyl_any    0  1  All
od                           
Overdose reported   2  1    3
not involved       35  4   39
All                37  5   42
Chi-square test statistic: 75.97300504695278
p-value: 0.841661729214757
df: 1


## Sensations Reported
---

**Among the 9 expected-cocaine samples that also contained fentanyl, 4 samples were reported as feeling  "normal," none weaker or stronger. Two samples were described as being "nice." No burning sensations, hallucinations, seizures, shorter/longer acting, up/down sensations, or skin issues were noted in free text descriptions.**

In [52]:
df_cocaine_fentanyl = df[(df['stimkind'] == 'Cocaine') & (df['lab_fentanyl_any'] == 1)]
df_cocaine_fentanyl[['sampleid', 'sensations', 'sen_strength', 'sen_weird', 'sen_hall', 'sen_up', 'sen_nice', 'sen_down', 'sen_long', 'sen_burn', 'sen_skin', 'sen_seizure']]

,sampleid,sensations,sen_strength,sen_weird,sen_hall,sen_up,sen_nice,sen_down,sen_long,sen_burn,sen_skin,sen_seizure
17,300265,normal,normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,300401,nice,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
47,300452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,300826,normal,normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,300896,normal; nice,normal,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
154,800181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,NC01011,normal,normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,NC01052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,NC01072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Physical description
---

**The 9 expected-cocaine samples that contained any fentanyl, were procuded in powder form (n=6/9 with 3 missing). One smaple was described as having a "dull" lustre. The colors described were white (n=5), gray (n=4), brown (n=1), off-white (n=1), and light pink (n=1).**

In [55]:
df_cocaine_fentanyl[['sampleid', 'crystals', 'powder', 'lustre', 'color', 'bright_color', 'lab_fentanyl', 'lab_fentanyl_any']]

,sampleid,crystals,powder,lustre,color,bright_color,lab_fentanyl,lab_fentanyl_any
17,300265,NaN,1.0,NaN,gray; brown,0,1,1
33,300401,NaN,1.0,NaN,white,0,1,1
47,300452,NaN,1.0,NaN,white,0,1,1
93,300826,NaN,1.0,NaN,off-white,0,1,1
102,300896,NaN,NaN,NaN,light pink; gray,0,1,1
154,800181,NaN,1.0,dull,white,0,0,1
161,NC01011,NaN,1.0,NaN,white,0,1,1
163,NC01052,NaN,NaN,NaN,gray,0,1,1
164,NC01072,NaN,NaN,NaN,white; gray,0,0,1


## Geograhic distribution
---

**Fentanyl-containing methamphetamine samples were observed in North Carolina, New York, and Washington.**

In [57]:
value_counts = df_cocaine_fentanyl['state'].value_counts()
print("States where cocaine-fentanyl was detected:")
print(value_counts)

States where cocaine-fentanyl was detected:
NC    5
NY    3
WA    1
Name: state, dtype: int64


## Text Comments
---

In [58]:
df_cocaine_fentanyl[['sampleid', 'texture_notes', 'sensation_notes']]

,sampleid,texture_notes,sensation_notes
17,300265,unclear,Unclear
33,300401,NaN,NaN
47,300452,NaN,NaN
93,300826,NaN,NaN
102,300896,NaN,NaN
154,800181,NaN,NaN
161,NC01011,NaN,NaN
163,NC01052,NaN,NaN
164,NC01072,NaN,NaN


# Expectation vs. Reality
---

**Sample donors displayed a high level of accuracy in being able to detect fentanyl in cocaine and methamphetamine (chi-sq 76, 1 df, p=<0.001). Out of 19 stimulant samples where an opioid was also suspected by the sample donor, lab analysis confirmed fentanyl in 79% (n=15). Conversely, fentanyl appeared in 5% of stimulant samples (8/159) unexpectedly.**

In [33]:
table = pd.crosstab(index=df['expect_opioid'], columns=df['lab_fentanyl_any'], margins=True)

print("Expectation vs. Reality Contingency Table:")
table.style.background_gradient(cmap='YlGn')

Expectation vs. Reality Contingency Table:


lab_fentanyl_any,0,1,All
expect_opioid,,,
0,151,8,159
1,4,15,19
All,155,23,178


In [34]:
table = pd.crosstab(index=df['expect_opioid'], columns=df['lab_fentanyl_any'])
chi2, p, dof, expected = chi2_contingency(table)
print("Chi-square test statistic:", chi2)
print("p-value:", p)
print("df:", dof)

Chi-square test statistic: 75.97300504695278
p-value: 2.8756926367136615e-18
df: 1


---
*fin.*